In [1]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [2]:
from pulp import *

In [3]:
nc = 6 #max number of color
n = 11 #number of nodes
V = range(1,n+1) #set of nodes
color = range(1,nc+1) #set of color

![](http://snag.gy/qCNdq.jpg)

In [4]:
E =   [[1, 2],
       [1, 5],
       [2, 4],
       [2, 5],
       [3, 6],
       [3, 8],
       [3, 7],
       [3, 4],
       [4, 5],
       [4, 6],
       [4, 7],
       [5, 7],
       [6, 8],
       [7, 8],
       [7, 9],
       [8, 10],
       [10, 11]]
# vertically ordered nodes len(E) == 17

In [5]:
# declare variables
# x[i,c] = 1 means that node i is assigned color c
x = LpVariable.dicts("x",(V,color), lowBound=0,upBound=1,cat=LpInteger)
all_possible_x = [(s,t) for s in V for t in color]

In [6]:
# y[c] = 1 means that color c is used, i.e. assigned to some node
y = LpVariable.dicts("y",color,0,1,LpInteger)

In [7]:
prob = LpProblem("minimum number of colors used",LpMinimize)
prob += lpSum([y[c] for c in color])

In [8]:
# each node must be assigned exactly one color
for i in V:
    prob += lpSum([x[i][c] for c in color]) == 1, "Node %s has been assigned one color"%i

In [9]:
# adjacent nodes cannot be assigned the same color
# because the index start with 0, so I have to do i-1
for (i,j) in E:
    for c in color:
        prob += x[i][c] + x[j][c] <= 1,"color %s isn't used in adjacent node %s and %s" %(c,i,j)

In [10]:
# make sure color is used 
for c in color:
    for i in V:
        prob += x[i][c]<= y[c]

In [11]:
prob.writeLP("color_min_problem.lp")

In [12]:
prob.solve()
print "Minimum number of color used = %s" %(value(prob.objective))

Minimum number of color used = 3.0


In [13]:
for v in V:
    print 'node %s' % v, 'use color',
    for c in color:
        if int(x[v][c].varValue) ==1:
            print c


node 1 use color 2
node 2 use color 5
node 3 use color 6
node 4 use color 2
node 5 use color 6
node 6 use color 5
node 7 use color 5
node 8 use color 2
node 9 use color 6
node 10 use color 6
node 11 use color 2


In [28]:


# A new LP problem
prob = LpProblem("dual", LpMaximize)

p1 = LpVariable("p1", 0)
p2 = LpVariable("p2", 0)
p3 = LpVariable("p3", 0)
v = LpVariable("v")


prob += v, "obj"

prob += -p2+2*p3+v<=0, "c1"
prob += p1-2*p3+v<=0,"c2"
prob += -2*p1+2*p2+v<=0,"c3"
prob += p1+p2+p3==1,"c4"

prob.solve()

print "Status:", LpStatus[prob.status]

for v in prob.variables():
	print v.name, "=", v.varValue, "\tReduced Cost =", v.dj

print "objective=", value(prob.objective)

print "\nSensitivity Analysis\nConstraint\t\tShadow Price\tSlack"
for name, c in prob.constraints.items():
	print name, ":", c, "\t", c.pi, "\t\t", c.slack

Status: Optimal
p1 = 0.4 	Reduced Cost = -0.0
p2 = 0.4 	Reduced Cost = -0.0
p3 = 0.2 	Reduced Cost = -0.0
v = 0.0 	Reduced Cost = -0.0
objective= 0.0

Sensitivity Analysis
Constraint		Shadow Price	Slack
c1 : -p2 + 2*p3 + v <= 0 	0.4 		-0.0
c2 : p1 - 2*p3 + v <= 0 	0.4 		-0.0
c3 : -2*p1 + 2*p2 + v <= 0 	0.2 		-0.0
c4 : p1 + p2 + p3 = 1 	-0.0 		-0.0


<bound method LpProblem.variables of dual:
MAXIMIZE
2*x1 + 3*x2 + -1*x4 + -2
SUBJECT TO
x5: - 9 x1 - 7 x2 + x4 >= -6

x6: 3 x1 - 9 x2 + x4 >= -2

x3: - x1 - x2 >= -1

VARIABLES
x1 Continuous
x2 Continuous
x4 Continuous
>